In [5]:
from keras.layers import Input, Dense, BatchNormalization, Conv2D, LeakyReLU, Lambda, Flatten, Reshape
from keras.models import Model
from keras.callbacks import TensorBoard, Callback
import h5py
import numpy as np
import math
import cmath
import time
from utils import *
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

num = 2
m = 16
n = 16
N_BS = m*n
N_ms = 1
cr = 3/8
mtx_v = N_BS
mtx_h = int(N_BS*cr)
output_dim = int(N_BS*cr)

SNR = [0, 5, 10, 15, 20]
# data loading
train = 'H_train.mat'
val = 'H_val.mat'
test = 'H_test.mat'
import scipy.io
# f = scipy.io.loadmat(train)

In [6]:
num = 2
m = 16
n = 16
N_BS = m*n
N_ms = 1
cr = 3/8
mtx_v = N_BS
mtx_h = int(N_BS*cr)
output_dim = int(N_BS*cr)

SNR = [0, 5, 10, 15, 20]
# data loading
train = 'H_train.mat'
val = 'H_val.mat'
test = 'H_test.mat'

mat = scipy.io.loadmat(train)
x_train = mat['H_train']
# x_train = np.transpose(x_train, [2, 1, 0])
x_train = x_train.astype('float32')  # 训练变量类型转换

mat = scipy.io.loadmat(val)
x_val = mat['H_val']
# x_val = np.transpose(x_val, [2, 1, 0])
x_val = x_val.astype('float32')  # 训练变量类型转换

mat = scipy.io.loadmat(test)
x_test = mat['H_test']
# x_test = np.transpose(x_test, [2, 1, 0])
x_test = x_test.astype('float32')  # 训练变量类型转换

In [39]:
mat = scipy.io.loadmat(train)
x_train = mat['H_train']
# x_train = np.transpose(x_train, [2, 1, 0])
x_train = x_train.astype('float32')  # 训练变量类型转换

In [42]:
mat = scipy.io.loadmat(train)

In [43]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Dec 06 20:59:31 2021',
 '__version__': '1.0',
 '__globals__': [],
 'H_train': array([[[-0.18649173, -0.06178995, -0.13575908, ..., -0.53297448,
          -0.19890173, -0.93904732],
         [-0.04555401, -0.08338627,  0.23937207, ..., -0.3059061 ,
          -0.08365838, -0.37805166]],
 
        [[-0.18656522, -0.06233173, -0.14947567, ..., -0.51706836,
          -0.19828873, -0.90770095],
         [-0.06311223, -0.0865146 ,  0.23274629, ..., -0.32831382,
          -0.08654447, -0.46782887]],
 
        [[-0.18457913, -0.06256368, -0.16274771, ..., -0.49901663,
          -0.19658102, -0.86477114],
         [-0.08061889, -0.08990589,  0.22537006, ..., -0.34968805,
          -0.08918391, -0.55311263]],
 
        ...,
 
        [[ 0.87411615,  0.86893041,  1.1944837 , ..., -1.42609207,
          -0.81955793, -5.86435073],
         [-0.03889684, -0.28846719, -0.4925879 , ..., -1.78246079,
          -4.97849229,  8.23310

In [44]:
x_train = mat['H_train']

In [45]:
x_train

array([[[-0.18649173, -0.06178995, -0.13575908, ..., -0.53297448,
         -0.19890173, -0.93904732],
        [-0.04555401, -0.08338627,  0.23937207, ..., -0.3059061 ,
         -0.08365838, -0.37805166]],

       [[-0.18656522, -0.06233173, -0.14947567, ..., -0.51706836,
         -0.19828873, -0.90770095],
        [-0.06311223, -0.0865146 ,  0.23274629, ..., -0.32831382,
         -0.08654447, -0.46782887]],

       [[-0.18457913, -0.06256368, -0.16274771, ..., -0.49901663,
         -0.19658102, -0.86477114],
        [-0.08061889, -0.08990589,  0.22537006, ..., -0.34968805,
         -0.08918391, -0.55311263]],

       ...,

       [[ 0.87411615,  0.86893041,  1.1944837 , ..., -1.42609207,
         -0.81955793, -5.86435073],
        [-0.03889684, -0.28846719, -0.4925879 , ..., -1.78246079,
         -4.97849229,  8.2331077 ]],

       [[ 0.90024762,  0.89447717,  1.22660021, ..., -1.41900949,
         -0.7835763 , -6.00311735],
        [ 0.04899949, -0.20158117, -0.3776372 , ..., -1.83299

In [40]:
x_train.shape

(512000, 2, 256)

In [41]:
512000/4000

128.0

In [28]:
mat = scipy.io.loadmat(test)
x_test = mat['H_test']
x_test.shape

(128000, 2, 256)

In [24]:
mat = scipy.io.loadmat(train)
x_train = mat['H_train']
x_train = np.transpose(x_train, [2, 1, 0])
x_train = x_train.astype('float32')

In [26]:
mat = scipy.io.loadmat(train)
x_train = mat['H_train']
x_train.shape

(512000, 2, 256)

In [22]:
x_test = np.transpose(x_test, [2, 1, 0])
x_train.shape

(256, 2, 512000)

In [7]:
def network(inputs, snr):
    y_tmp = Dense(output_dim, activation='linear', use_bias=False)(inputs)
    # y_tmp = MatMul(output_dim, mtx_v, mtx_h)(inputs)
    y = Lambda(get_noise, arguments={'snr': snr})(y_tmp)
    decode = Dense(N_BS, activation='linear', use_bias=False)(y)
    # decode = MatMul(dictionary_dim, dictionary_mtx_v, dictionary_mtx_h)(y)
    h_tmp = Reshape((num, m, n))(decode)
    h = residual_block_decoded(h_tmp)
    h = Reshape((num, N_BS))(h)
    # h = Dense(N_BS, activation='linear')(h)
    # h = Lambda(dic, arguments={'P_real_init': P_real, 'P_imag_init':P_imag})(h)
    return h

In [8]:
snr=0
inpt = Input(shape=(num, N_BS,))
outpt = network(inpt, snr)
model = Model(inputs=[inpt], outputs=[outpt])
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2, 256)]     0           []                               
                                                                                                  
 dense (Dense)                  (None, 2, 96)        24576       ['input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 2, 96)        0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 2, 256)       24576       ['lambda[0][0]']                 
                                                                                              

In [9]:
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses_train = []
        self.losses_val = []

    def on_batch_end(self, batch, logs={}):
        self.losses_train.append(logs.get('loss'))

    def on_epoch_end(self, epoch, logs={}):
        self.losses_val.append(logs.get('val_loss'))

In [10]:
history = LossHistory()
file = 'model_new_D' + '_cr_' + str(cr) + '_snr_' + str(snr)
path = 'result_375/TensorBoard_%s' % file

In [11]:
model.fit(x_train, x_train,
          epochs=300,
          batch_size=128,
          shuffle=True,
          validation_data=(x_val, x_val),
          callbacks=[history,
                     TensorBoard(log_dir=path)])

C:\Users\admin\anaconda3\envs\Tensorflow2\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/300
4000/4000 [==============================] - 92s 22ms/step - loss: 0.2097 - val_loss: 0.1713
Epoch 2/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1638 - val_loss: 0.1639
Epoch 3/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1582 - val_loss: 0.1618
Epoch 4/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1551 - val_loss: 0.1612
Epoch 5/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1529 - val_loss: 0.1577
Epoch 6/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1513 - val_loss: 0.1568
Epoch 7/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1501 - val_loss: 0.1570
Epoch 8/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1492 - val_loss: 0.1564
Epoch 9/300
4000/4000 [==============================] - 85s 21ms/step - loss: 0.1483 - val_loss: 0.1553
Epoch 10/300
4000/4000 [==============================]

4000/4000 [==============================] - 91s 23ms/step - loss: 0.1390 - val_loss: 0.1510
Epoch 77/300
4000/4000 [==============================] - 90s 22ms/step - loss: 0.1388 - val_loss: 0.1514
Epoch 78/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1389 - val_loss: 0.1510
Epoch 79/300
4000/4000 [==============================] - 86s 21ms/step - loss: 0.1388 - val_loss: 0.1503
Epoch 80/300
4000/4000 [==============================] - 88s 22ms/step - loss: 0.1387 - val_loss: 0.1502
Epoch 81/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1388 - val_loss: 0.1508
Epoch 82/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1387 - val_loss: 0.1508
Epoch 83/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1387 - val_loss: 0.1507
Epoch 84/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1387 - val_loss: 0.1510
Epoch 85/300
4000/4000 [==============================] - 8

4000/4000 [==============================] - 89s 22ms/step - loss: 0.1371 - val_loss: 0.1510
Epoch 153/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1370 - val_loss: 0.1512
Epoch 154/300
4000/4000 [==============================] - 89s 22ms/step - loss: 0.1370 - val_loss: 0.1508
Epoch 155/300
4000/4000 [==============================] - 90s 22ms/step - loss: 0.1370 - val_loss: 0.1514
Epoch 156/300
4000/4000 [==============================] - 90s 22ms/step - loss: 0.1369 - val_loss: 0.1506
Epoch 157/300
4000/4000 [==============================] - 87s 22ms/step - loss: 0.1369 - val_loss: 0.1509
Epoch 158/300
4000/4000 [==============================] - 90s 23ms/step - loss: 0.1369 - val_loss: 0.1510
Epoch 159/300
4000/4000 [==============================] - 90s 23ms/step - loss: 0.1369 - val_loss: 0.1505
Epoch 160/300
4000/4000 [==============================] - 86s 21ms/step - loss: 0.1369 - val_loss: 0.1511
Epoch 161/300
4000/4000 [==========================

4000/4000 [==============================] - 91s 23ms/step - loss: 0.1361 - val_loss: 0.1518
Epoch 229/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1360 - val_loss: 0.1512
Epoch 230/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1360 - val_loss: 0.1509
Epoch 231/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1360 - val_loss: 0.1514
Epoch 232/300
4000/4000 [==============================] - 90s 23ms/step - loss: 0.1361 - val_loss: 0.1507
Epoch 233/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1360 - val_loss: 0.1508
Epoch 234/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1360 - val_loss: 0.1520
Epoch 235/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1360 - val_loss: 0.1516
Epoch 236/300
4000/4000 [==============================] - 91s 23ms/step - loss: 0.1359 - val_loss: 0.1512
Epoch 237/300
4000/4000 [==========================

In [12]:
tStart = time.time()
x_hat = model.predict(x_test)
tEnd = time.time()
print("It cost %f sec" % ((tEnd - tStart) / x_test.shape[0]))

x_re = x_test[:, 0, :]
x_im = x_test[:, 1, :]
x_test_c = x_re + 1j * x_im

x_hat_re = x_hat[:, 0, :]
x_hat_im = x_hat[:, 1, :]
x_hat_c = x_hat_re + 1j * x_hat_im

power = np.sum(abs(x_test_c) ** 2, axis=1)
mse = np.sum(abs(x_test_c - x_hat_c) ** 2, axis=1)

print("When compress rate is", cr)
print('SNR is', snr, 'dB')
print("NMSE is ", 10 * math.log10(np.mean(mse / power)), 'dB')
'''
filename = "result/model_%s.csv" % file
x_hat1 = np.reshape(x_hat, (len(x_hat), -1))
np.savetxt(filename, x_hat1, delimiter=",")
'''
model_json = model.to_json()
outfile = "result_375/saved_model/%s.json" % file
with open(outfile, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
outfile = "result_375/saved_model/%s.h5" % file
model.save_weights(outfile)

It cost 0.000113 sec
When compress rate is 0.375
SNR is 0 dB
NMSE is  -5.033408228199624 dB


FileNotFoundError: [Errno 2] No such file or directory: 'result_375/saved_model/model_new_D_cr_0.375_snr_0.json'

In [13]:
model_json = model.to_json()
outfile = "result_375/saved_model/%s.json" % file
with open(outfile, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
outfile = "result_375/saved_model/%s.h5" % file
model.save_weights(outfile)

In [24]:
x_re = x_test[:, 0, :]
x_im = x_test[:, 1, :]
x_test_c = x_re + 1j * x_im

In [33]:
x_test_c.shape

(128000, 256)

In [34]:
power = np.sum(abs(x_test_c) ** 2, axis=0)

In [35]:
power

array([135803.67   , 125208.99   , 120958.81   , 172062.06   ,
       148947.42   , 216666.75   , 352411.3    ,  36361.492  ,
        14966.121  ,  27756.934  , 227355.61   , 263872.97   ,
       163634.77   , 130533.03   , 163753.95   , 176087.27   ,
       106749.55   , 140258.58   , 182236.53   , 156282.52   ,
       233533.61   , 285707.72   , 273399.4    ,  31674.71   ,
        15849.468  ,  32319.727  , 256499.28   , 227838.66   ,
       192964.48   , 159700.16   , 212183.8    , 106817.3    ,
       169967.77   , 117781.66   , 131232.42   , 212068.72   ,
       276504.97   , 357342.2    , 316745.75   ,  38707.21   ,
        16907.428  ,  31814.375  , 243307.5    , 386589.97   ,
       188397.77   , 145539.     , 236014.94   , 151589.58   ,
       173698.5    , 120823.9    , 175691.62   , 234501.53   ,
       275756.06   , 249761.81   , 282376.22   ,  35174.35   ,
        18439.365  ,  39893.383  , 411013.3    , 281122.8    ,
       301631.3    , 172372.89   , 118842.69   , 136293

In [27]:
mse = np.sum(abs(x_test_c - 0) ** 2, axis=1)

In [31]:
x_re = x_test[:, 0, :]

In [38]:
x_re.transpose().shape

(256, 128000)

In [18]:
from keras import backend as k
from keras.models import model_from_json
import h5py
import scipy.io as sio
import numpy as np
import math
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from utils import *
cr = 1/2
NMSE = []
SNR = [0, 5, 10, 15, 20]
# SNR = [0, 5, 15, 20]
snr=0
mat = scipy.io.loadmat(test)
x_test = mat['H_test']
# x_test = np.transpose(x_test, [2, 1, 0])
x_test = x_test.astype('float32')  # 训练变量类型转换
# test = 'old_375/H_test.mat'
# mat = h5py.File(test)
# x_test = mat['H_test']
# x_test = np.transpose(x_test, [2, 1, 0])
# x_test = x_test.astype('float32')  # 训练变量类型转换

In [22]:
cr=0.375

In [23]:


file = 'model_new_D' + '_cr_' + str(cr) + '_snr_' + str(snr)
outfile = "result_375/saved_model/%s.json" % file
# outfile = "old_375/%s.json" % file
json_file = open(outfile, 'r')
loaded_model_json = json_file.read()
json_file.close()
net = model_from_json(loaded_model_json, custom_objects={'k': k})
outfile = "result_375/saved_model/%s.h5" % file
# outfile = "old_375/%s.h5" % file
net.load_weights(outfile)

x_hat = net.predict(x_test)

x_re = x_test[:, 0, :]
x_im = x_test[:, 1, :]
x_test_c = x_re + 1j * x_im

x_hat_re = x_hat[:, 0, :]
x_hat_im = x_hat[:, 1, :]
x_hat_c = x_hat_re + 1j * x_hat_im

power = np.sum(abs(x_test_c) ** 2, axis=1)
mse = np.sum(abs(x_test_c - x_hat_c) ** 2, axis=1)
nmse = 10*math.log10(np.mean(mse/power))
print('SNR is', snr, 'dB')
print("NMSE is ", nmse, 'dB')
NMSE.append(nmse)

sio.savemat('NMSE_DL_4_path_cr_'+ str(cr) + '.mat', mdict={'NMSE_DL_4_path_2': NMSE})

SNR is 0 dB
NMSE is  -5.031026136367213 dB
